# IPython & Jupyter: a PyCon 2017 tutorial

In [53]:
#! pip install pymystem3

from pymystem3 import Mystem
import pandas as pd

In [54]:
url = 'https://raw.githubusercontent.com/astafyevai/python_final.project/main/masterimargarita.txt'

In [55]:
import requests

response = requests.get(url)
text = response.text

In [56]:
data[:1000]

'Мастер и Маргарита\nМихаил Афанасиевич Булгаков\n\nРусская классика\n\n  \xa0 Прекрасно оформленное подарочное издание в шелковом переплете с трехсторонним окрашенным обрезом. Обложка книги украшена аппликацией. В книге представлено бессмертное творение русской литературы XX века. Легендарная фантасмагория "Мастер и Маргарита"... Произведения Михаила Булгакова - многоплановые, неоднозначные, раскрывающие разные грани его яркого таланта! Глубоко философский роман М.Булгакова иллюстрациями П.Оринянского не исчерпан, но и не умален. Иллюстрации обогатили книгу прекрасными портретами-образами, сделали интересней и для чтения, и для зрительского восприятия и, пожалуй, самое главное, - открыли новое духовное пространство. Художник вслед за Булгаковым ищет путь не к логике, а к интуиции читателя, что-то нашептывает ему, подсказывает, намекает, помогает ему ощутить сладость самостоятельных читательских открытий. Силовые линии иллюстраций П.Оринянского обращены не внутрь листа, а рвутся наружу

In [58]:
m = Mystem()

In [59]:
lemmas = m.lemmatize(text)
lemmas[10:20]

['булгаков',
 '\n',
 '\n',
 'русский',
 ' ',
 'классика',
 '\n',
 '\n',
 '  \xa0 ',
 'прекрасно']

In [60]:
p = ([x for x in lemmas if x.isalpha()])

In [61]:
len(p)

112106

In [62]:
pd.Series(p).nunique()

10418

In [63]:
pd.Series(p).value_counts().head(50)

и              5025
в              3651
он             2167
не             2031
на             2005
что            1876
я              1391
быть           1388
с              1306
а               977
она             886
как             841
это             800
то              784
вы              761
этот            760
но              714
к               701
маргарита       624
же              595
у               528
из              528
по              478
сказать         470
за              469
ты              426
тот             423
так             411
свой            411
от              382
один            374
рука            368
все             366
они             361
который         355
весь            355
иван            354
человек         349
глаз            346
отвечать        341
о               327
тут             306
да              306
себя            302
бы              300
его             298
становиться     285
только          281
уже             278
нет             274


In [72]:
ana = m.analyze(','.join(list(set(p))))


In [87]:
#i = 0
#for w in ana:
#    i += 1
#    if w['text'] == 'воланд':
#        print(i)

14255


In [88]:
# ana[14254]

{'analysis': [{'lex': 'воланд',
   'wt': 0.7324094467,
   'gr': 'S,фам,муж,од=им,ед'}],
 'text': 'воланд'}

In [81]:
words_include = []
for word in ana:
    if ('analysis' in word) and word['analysis']:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        if pos == 'S' and 'имя' not in gr and 'фам' not in gr and 'отч' not in gr and 'сокр' not in gr:
            words_include.append(word['text'])

In [76]:
#p_new = set(p) - set(words_exclude)

In [82]:
p_new = [x for x in p if x in words_include]

In [83]:
pd.Series(p_new).value_counts().head(15)

рука          368
человек       349
глаз          346
прокуратор    267
мочь          249
голова        246
кот           233
лицо          230
время         214
голос         184
дело          167
мастер        167
слово         159
гость         155
квартира      152
dtype: int64

In [84]:
names_include = []
for word in ana:
    if ('analysis' in word) and word['analysis']:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        if pos == 'S' and ('имя' in gr or 'фам' in gr):
            names_include.append(word['text'])

In [85]:
names = [x for x in p if x in names_include]

In [86]:
pd.Series(names).value_counts().head(15)

маргарита    624
иван         354
воланд       266
коровьев     227
пилат        196
азазелло     152
берлиоз      147
никанор      134
варенуха     103
степа        101
левий         75
иешуа         58
наташа        53
аннушка       52
бездомный     46
dtype: int64

Class outline:

* A quick installation check of [ipython](https://ipython.org/install.html) and [jupyter notebook](https://jupyter.readthedocs.io/en/latest/install.html)
* An overview of the IPython project from [the official website](http://ipython.org), and [jupyter](https://jupyter.org)
* Super basic intro to the notebook: typing code.
* [Notebook Basics](examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Customizing IPython - a condensed version](exercises/Customization/Condensed.ipynb)
* [Running a Secure Public Notebook Server](examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter/IPython works](examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

Get this tutorial:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.continuum.io/downloads):

    conda install ipython jupyter

with pip:

    # first, upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook

There are a lot more detailed notebooks in this same directory that cover other topics, but we can not cover all in a 3-hour tutorial. We encourage you to explore them and practice on your own.